In [144]:
import torch
import torchvision
from torch.utils.data.dataset import Dataset  # for data-sets
from torchvision import transforms
from PIL import Image
import glob
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import numpy as np


In [145]:
# get all the image and mask

folder_data = np.load("all_slices_image_1channel.npy")
folder_mask = np.load("all_slices_mask_1channel.npy")
print(type(folder_data))

# split the data into train and test
len_data = len(folder_data)
print('len_data',len_data)
train_size = 0.8

#move the slices into ararys for train(img+msk) and test(img+msk)
train_image_npy = folder_data[:int(len_data*train_size)]
train_mask_npy = folder_mask[:int(len_data*train_size)]

validation_image_npy = folder_data[int(len_data*train_size):]
validation_mask_npy = folder_mask[int(len_data*train_size):]

class CustomDataset(Dataset):
    def __init__(self, image_npy, mask_npy, train=True):   # initial logic happens like transform
        
        self.image_npy = image_npy
        self.mask_npy = mask_npy
        self.transforms  = transforms.Compose([transforms.ToTensor(), transforms.ToPILImage(), transforms.ToTensor()])

    def __getitem__(self, index):
       # print('Index',index)
        self.image = np.asarray(self.image_npy[index])
        self.mask = np.asarray(self.mask_npy[index])
      #  print('Before converting into tensor image and mask shape', self.image.shape ,  self.image.shape)
        self.image = self.transforms(self.image)
        self.mask = self.transforms(self.mask)
       # print('After Image and Mask Shape',self.image.shape, self.mask.shape)
        return self.image, self.mask

    def __len__(self):  # return count of sample we have

        return len(self.image_npy)

batch_size = 5
train_dataset = CustomDataset(train_image_npy, train_mask_npy, train=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=4)

validation_dataset = CustomDataset(validation_image_npy, validation_mask_npy, train=False)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, num_workers=4)


<class 'numpy.ndarray'>
len_data 11000


In [146]:
dataloaders = {
    'train': train_loader,
    'val': validation_loader
}

In [147]:
len(train_dataset)

8800

In [148]:
print(len(train_dataset[8799]))
print('Image',train_dataset[8799][0].shape)
print('Mask', train_dataset[8799][1].shape)

2
Image torch.Size([1, 256, 256])
Mask torch.Size([1, 256, 256])


In [126]:
inputs, masks = next(iter(dataloaders['train']))

In [139]:
#from torchsummary import summary
import torch
import torch.nn as nn
#import pytorch_unet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from model import UNet
model = UNet().to(device)
loss_fun = torch.nn.BCELoss()

In [142]:
from collections import defaultdict
import torch.nn.functional as F
from dice_loss import dice_coeff

def calc_loss(pred, target, metrics, bce_weight=0.5):  
    bce = F.binary_cross_entropy_with_logits(pred, target) 
    dice = dice_coeff(pred, target)
  #  print(bce)
    #print('bce', type(bce))       
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    
   # print('dice', type(dice))
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
   # metrics['loss'] += loss.data.cpu().numpy() * target.size(0)   
    return bce

def print_metrics(metrics, epoch_samples, phase):    
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {}".format(str(k), str(metrics[k] / epoch_samples)))
    
    print( (" {}: {} ").format(phase, ", ".join(outputs)))    

def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    best_dice = 0.5
       
    for epoch in range(num_epochs): #25
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)    
        since = time.time()     
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])                    
                model.train()  # Set model to training mode
                
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    if phase == 'train':
                        i=0
                    else:
                        i=2
                    loss = calc_loss(outputs, labels, metrics)                  
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['bce'] / epoch_samples
            epoch_dice = metrics['dice'] / epoch_samples

            # deep copy the model
            if phase == 'val' and best_dice < epoch_dice:
                print("saving best model")
                best_dice = epoch_dice
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_dice))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [143]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 2

from model import UNet
model = UNet().to(device)

# Adam Optimizer
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)
#opt = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma= 0.1)
model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=30)

cuda:0
Epoch 0/29
----------
LR 0.0001
 train: bce: 0.6971829482777552, dice: [0.01023416] 
 val: bce: 0.6931471824645996, dice: [5.721024e-09] 
1m 44s
Epoch 1/29
----------
LR 0.0001
 train: bce: 0.6931471824645996, dice: [5.7182223e-09] 
 val: bce: 0.6931471824645996, dice: [5.721024e-09] 
1m 46s
Epoch 3/29
----------
LR 0.0001
 train: bce: 0.6931471824645996, dice: [5.7182223e-09] 
 val: bce: 0.6931471824645996, dice: [5.721024e-09] 
1m 46s
Epoch 4/29
----------
LR 0.0001
 train: bce: 0.6931471824645996, dice: [5.7182223e-09] 
 val: bce: 0.6931471824645996, dice: [5.721024e-09] 
1m 46s
Epoch 5/29
----------
LR 1e-05
 train: bce: 0.6931471824645996, dice: [5.7182223e-09] 
 val: bce: 0.6931471824645996, dice: [5.721024e-09] 
1m 46s
Epoch 6/29
----------
LR 1e-05
 train: bce: 0.6931471824645996, dice: [5.7182223e-09] 
 val: bce: 0.6931471824645996, dice: [5.721024e-09] 
1m 46s
Epoch 7/29
----------
LR 1e-05
 train: bce: 0.6931471824645996, dice: [5.7182223e-09] 
 val: bce: 0.6931471824